In [ ]:
# @title
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import os


In [ ]:
# # @title
# from google.colab import drive
# drive.mount('/content/Drive')

In [ ]:
# taking input of the file
csv_path = "/content/nifty50_1min_2015_to_2024.csv"
base_data = pd.read_csv(csv_path)
df = base_data.copy()

# ## creating a sample dataframe so that I can dry run the code for leser computation
# sample_point = len(df)//5
# df_sample = df[:sample_point]

# Create a clean copy and handle datetime with timezone
df = df.copy()
df.loc[:, 'date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

# Resample to 5min (preserving timezone)
df_5min = df.resample('5min', closed='left', label='left').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()

# If you need to remove timezone, uncomment the next line:
# df_5min.index = df_5min.index.tz_localize(None)

print("Original shape:", df.shape)
print("New 5-min shape:", df_5min.shape)
print("\nFirst few rows of 5-min data:")
print(df_5min.head())


# Calculate returns while preserving the index
returns_df = pd.DataFrame()
returns_df['returns'] = df_5min['close'].pct_change()
returns_df = returns_df.dropna()

print("Shape of returns dataframe:", returns_df.shape)
print("\nFirst few returns (should show % changes between the close prices we saw):")
print(returns_df.head())

Original shape: (890511, 5)
New 5-min shape: (178118, 5)

First few rows of 5-min data:
                              open     high      low    close  volume
date                                                                 
2015-01-09 09:15:00+05:30  8285.45  8301.30  8285.45  8301.20       0
2015-01-09 09:20:00+05:30  8300.50  8303.00  8293.25  8301.00       0
2015-01-09 09:25:00+05:30  8301.65  8302.55  8286.80  8294.15       0
2015-01-09 09:30:00+05:30  8294.10  8295.75  8280.65  8288.50       0
2015-01-09 09:35:00+05:30  8289.10  8290.45  8278.00  8283.45       0
Shape of returns dataframe: (178117, 1)

First few returns (should show % changes between the close prices we saw):
                            returns
date                               
2015-01-09 09:20:00+05:30 -0.000024
2015-01-09 09:25:00+05:30 -0.000825
2015-01-09 09:30:00+05:30 -0.000681
2015-01-09 09:35:00+05:30 -0.000609
2015-01-09 09:40:00+05:30  0.000254


/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [ ]:
# @title
# Add this at the start of your notebook
def setup_wandb():
    try:
        import wandb
        # Check if already logged in
        if wandb.api.api_key is None:
            # Your API key
            WANDB_API_KEY = "641b305133f7d8345e710ecf6c9d83fea7e225f1"
            os.environ["WANDB_API_KEY"] = WANDB_API_KEY

        print("WandB setup complete!")
        return True
    except Exception as e:
        print(f"Error setting up WandB: {str(e)}")
        return False

# Use it in your notebook
setup_wandb()

WandB setup complete!


True

In [ ]:
####### Libraries -
# @title
import pandas as pd
import numpy as np
import wandb
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
from functools import lru_cache
from numba import jit
from tqdm.auto import tqdm
import time
import cupy as cp
from concurrent.futures import ProcessPoolExecutor, as_completed
import logging
import traceback


# Configure the logger
def setup_logger():
    # Create a logger
    logger = logging.getLogger("MyLogger")
    logger.setLevel(logging.DEBUG)  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)

    # Create a file handler
    file_handler = logging.FileHandler("application.log")  # Specify the log file name
    file_handler.setLevel(logging.DEBUG)  # Set the logging level for the handler

    # Create a formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)  # Attach the formatter to the handler

    # Add the handler to the logger
    logger.addHandler(file_handler)

    return logger

#### this function is being used in plot regimes and process single combination
@jit(nopython=True)
def convert_days_to_periods(days, timeframe=5):
    """Numba optimized version"""
    minutes_per_day = 375  # Trading hours
    periods_per_day = minutes_per_day // timeframe
    return days * periods_per_day

### this function is used in bai perron test
@jit(nopython=True)
def calculate_aic(rss, n_params, n_samples):
    """Numba optimized AIC calculation"""
    return n_samples * np.log(rss/n_samples) + 2 * n_params

### this function is used in bai perron test
@lru_cache(maxsize=200)
def cached_ols_fit(data_key, X_key):
    """
    Cached OLS fitting with GPU support and error handling

    Parameters:
    -----------
    data_key : tuple
        Data array converted to tuple for caching
    X_key : tuple
        Design matrix converted to tuple for caching

    Returns:
    --------
    OLS fitted model
    """
    try:
        # Convert to numpy arrays
        data = np.array(data_key)
        X = np.array(X_key)

        # Check dimensions
        if X.shape[0] != data.shape[0]:
            raise ValueError("Data and design matrix dimensions mismatch")

        # Fit OLS model
        model = OLS(data, X).fit()

        return model

    except Exception as e:
        print(f"Error in OLS fitting: {e}")
        raise e


### this function is used in process single combination
def bai_perron_test(data, max_breaks=5, min_size=0.1, early_stopping_patience=3):
    start_time = time.time()
    processing_times = {}
    try:
        # Input validation with timing
        validation_start = time.time()
        if len(data) < 100:
            raise ValueError("Data length must be at least 100 points")
        if max_breaks < 1:
            raise ValueError("max_breaks must be at least 1")
        if not (0 < min_size < 0.5):
            raise ValueError("min_size must be between 0 and 0.5")
        processing_times['validation'] = time.time() - validation_start

        print(f"Starting analysis with data length: {len(data)}")
        print(f"Parameters: max_breaks={max_breaks}, min_size={min_size}")

        # Data prep
        data_prep_start = time.time()
        data_gpu = cp.asarray(data, dtype=cp.float32)
        n = len(data_gpu)
        min_size_obs = int(min_size * n)
        processing_times['data_preparation'] = time.time() - data_prep_start
        print(f"Data preparation completed. Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")

        # Optimized Array Initialization with timing
        array_init_start = time.time()
        rss_array = cp.zeros((n, max_breaks+1), dtype=cp.float32)
        break_points = cp.zeros((n, max_breaks+1), dtype=cp.int32)
        aic_scores = cp.zeros(max_breaks+1, dtype=cp.float32)
        processing_times['array_initialization'] = time.time() - array_init_start

        # Base case calculation with timing
        base_calc_start = time.time()
        print("\nStarting base case calculations...")

        X_gpu = cp.column_stack((
            cp.ones(n, dtype=cp.float32),
            cp.arange(n, dtype=cp.float32)
        ))

        # Calculate base case (no breaks)

        beta_base = cp.linalg.solve(X_gpu.T @ X_gpu, X_gpu.T @ data_gpu)
        residuals = data_gpu - X_gpu @ beta_base
        base_rss = float(cp.sum(residuals**2))
        rss_array[0, 0] = base_rss
        aic_scores[0] = calculate_aic(base_rss, 2, n)

        processing_times['base_case'] = time.time() - base_calc_start
        print(f"Base case completed:")
        print(f"- Base RSS: {base_rss:.4e}")
        print(f"- Initial AIC: {float(aic_scores[0]):.4e}")
        print(f"- Time taken: {processing_times['base_case']:.2f} seconds")
        print(f"- Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")

        # Batch Processing for Large Datasets
        batch_start_time = time.time()
        batch_size = min(2000, n//20)
        no_improve = 0
        best_aic = float('inf')
        total_batches = ((n-min_size_obs+1) - min_size_obs) // batch_size + 1

        print(f"\nStarting batch processing:")
        print(f"- Batch size: {batch_size}")
        print(f"- Total batches per break: {total_batches}")
        print(f"- Memory at start: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")
        print(f"- Min size observations: {min_size_obs}")
        print(f"- Expected iterations: {total_batches * max_breaks}")
        print(f"- Data range: index {min_size_obs} to {n-min_size_obs}")

        with tqdm(total=max_breaks, desc="Break Detection", position=0, leave=True) as pbar:
            for m in range(1, max_breaks+1):
                break_start_time = time.time()
                min_aic = float('inf')
                best_break = None
                batch_count = 0

                # Process in batches with nested progress bar
                with tqdm(total=total_batches, desc=f"Break {m}", position=1, leave=False) as batch_pbar:
                    for start_idx in range(min_size_obs, n-min_size_obs+1, batch_size):
                        batch_count += 1
                        end_idx = min(start_idx + batch_size, n-min_size_obs+1)

                        print(f"\nProcessing Batch {batch_count}/{total_batches}:")
                        print(f"Range: {start_idx} to {end_idx}")
                        print(f"Current memory: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")
                        # Process batch
                        with tqdm(total=end_idx-start_idx,
                        desc=f"Processing indices {start_idx}-{end_idx}",
                        position=2,
                        leave=False,
                        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {postfix}') as inner_pbar:

                          for i in range(start_idx, end_idx):
                              # Memory management
                              if i % 300 == 0:  # More frequent cleanup
                                cp.cuda.Stream.null.synchronize()
                                mempool = cp.get_default_memory_pool()
                                mempool.free_all_blocks()
                                print(f"\nMemory cleared at index {i} - Current usage: {mempool.used_bytes() / 1e9:.2f} GB")

                              y1, y2 = data_gpu[:i], data_gpu[i:]
                              X1, X2 = X_gpu[:i], X_gpu[i:]

                              # Convert to tuples for caching
                              y1_tuple = tuple(cp.asnumpy(y1))
                              y2_tuple = tuple(cp.asnumpy(y2))
                              X1_tuple = tuple(map(tuple, cp.asnumpy(X1)))
                              X2_tuple = tuple(map(tuple, cp.asnumpy(X2)))

                              if i % 300 == 0:  # Print every 100 iterations
                                print(f"\nProgress - Index {i}/{end_idx}:")
                                print(f"Segment sizes - Left: {y1.shape[0]}, Right: {y2.shape[0]}")
                                print(f"Memory: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")
                                inner_pbar.set_postfix({
                                    'Current AIC': f'{current_aic:.2f}' if 'current_aic' in locals() else 'N/A',
                                    'Best AIC': f'{min_aic:.2f}'
                                })

                              try:
                                  # Use cached OLS fitting
                                  model1 = cached_ols_fit(y1_tuple, X1_tuple)
                                  model2 = cached_ols_fit(y2_tuple, X2_tuple)

                                  # Get predictions
                                  residuals1 = y1 - cp.asarray(model1.predict(cp.asnumpy(X1)))
                                  residuals2 = y2 - cp.asarray(model2.predict(cp.asnumpy(X2)))

                                  rss = float(cp.sum(residuals1**2) + cp.sum(residuals2**2))
                                  current_aic = calculate_aic(rss, 2*(m+1), n)

                                  if current_aic < min_aic:
                                      min_aic = current_aic
                                      best_break = i

                              except Exception as e:
                                  print(f"\nError at break {m}, index {i}:")
                                  print(f"- Error type: {type(e).__name__}")
                                  print(f"- Error message: {str(e)}")
                                  print(f"- Data shapes: y1={y1.shape}, y2={y2.shape}")
                                  print(f"- Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")
                                  print("Continuing to next index...")
                                  continue


                              inner_pbar.update(1)

                                  # Memory cleanup every 100 iterations
                              if i % 300 == 0:
                                cp.cuda.Stream.null.synchronize()
                                mempool = cp.get_default_memory_pool()
                                mempool.free_all_blocks()

                        batch_pbar.update(1)
                        batch_pbar.set_postfix({
                          'Best AIC': f'{min_aic:.2f}',
                          'Best Break': best_break,
                          'Memory': f'{cp.get_default_memory_pool().used_bytes() / 1e9:.2f}GB'
                      })
                # Early stopping with accuracy preservation
                if min_aic > best_aic:
                    no_improve += 1
                    if no_improve >= early_stopping_patience:
                        print("\nEarly stopping triggered")
                        break
                else:
                    best_aic = min_aic
                    no_improve = 0

                ## Memory status after each break attempt
                current_memory = cp.get_default_memory_pool().used_bytes() / 1e9
                print(f"Memory after break {m}: {current_memory:.2f} GB")
                if current_memory > 2.0:  # If memory usage is high
                    print("High memory usage detected - cleaning memory...")
                    cp.cuda.Stream.null.synchronize()
                    mempool = cp.get_default_memory_pool()
                    mempool.free_all_blocks()

                if best_break is not None:
                    break_points[:, m] = best_break
                    aic_scores[m] = min_aic

                break_time = time.time() - break_start_time
                print(f"\nBreak {m} completed:")
                print(f"- Time taken: {break_time:.2f} seconds")
                print(f"- Best AIC: {min_aic:.2f}")
                print(f"- Best break point: {best_break} ({best_break/n:.2%} of data)")
                print(f"- Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")
                print(f"- No improvement count: {no_improve}/{early_stopping_patience}")

                pbar.update(1)
                pbar.set_postfix({
                    'AIC': f'{min_aic:.2f}',
                    'Breaks Found': f'{m}/{max_breaks}',
                    'Memory': f'{cp.get_default_memory_pool().used_bytes() / 1e9:.2f}GB'
                })

        processing_times['batch_processing'] = time.time() - batch_start_time

        # Results Processing and Memory Cleanup
        results_start_time = time.time()
        print("\nProcessing final results...")

        # Find optimal breaks
        optimal_breaks = int(cp.argmin(aic_scores))
        break_dates = sorted([
            int(break_points[i, i+1])
            for i in range(optimal_breaks)
        ])

        print(f"\nResults summary:")
        print(f"- Optimal number of breaks: {optimal_breaks}")
        print(f"- Break points: {break_dates}")
        print(f"- Final AIC score: {float(aic_scores[optimal_breaks]):.4e}")

        # Memory cleanup
        print("\nCleaning up GPU memory...")
        mempool = cp.get_default_memory_pool()
        pinned_mempool = cp.get_default_pinned_memory_pool()
        initial_memory = mempool.used_bytes()

        mempool.free_all_blocks()
        pinned_mempool.free_all_blocks()

        final_memory = mempool.used_bytes()
        memory_freed = initial_memory - final_memory

        print(f"- Memory freed: {memory_freed / 1e9:.2f} GB")

        # Timing summary
        processing_times['results_processing'] = time.time() - results_start_time
        total_time = time.time() - start_time

        print("\nTiming Summary:")
        for step, duration in processing_times.items():
            print(f"- {step}: {duration:.2f} seconds")
        print(f"- Total execution time: {total_time:.2f} seconds")

        return break_dates, cp.asnumpy(aic_scores)

    except Exception as e:
        print(f"Error in bai_perron_test: {str(e)}")
        traceback.print_exc()
        raise e


## this function is used in process single combination
def analyze_regime_characteristics(data, break_dates):
    processing_times = {}
    start_time = time.time()

    # Initialize data storage lists
    regime_data = []
    regime_returns_data = []

    # Validate inputs
    validation_start = time.time()
    try:
        if not isinstance(break_dates, (list, np.ndarray)):
            raise ValueError("break_dates must be a list or numpy array")
        if len(break_dates) == 0:
            print("Warning: No break dates provided, treating entire series as one regime")
            break_dates = []
    except Exception as e:
        print(f"Error in input validation: {e}")
        raise
    processing_times['validation'] = time.time() - validation_start

    print(f"\nStarting regime analysis:")
    print(f"- Number of breaks: {len(break_dates)}")
    print(f"- Data length: {len(data)}")

    # Initialize timing and calculation helpers
    calc_start_time = time.time()
    data_series = pd.Series(data)  # Convert once instead of multiple times

    # Cache rolling calculations
    vol_5d = data_series.rolling(5).std()
    vol_20d = data_series.rolling(20).std()

    processing_times['preparation'] = time.time() - calc_start_time
    print(f"Data preparation completed in {processing_times['preparation']:.2f} seconds")

    try:
        with tqdm(total=len(break_dates) + 1, desc="Analyzing Regimes") as pbar:
            start_idx = 0

            for regime_num, break_idx in enumerate(break_dates + [len(data)], 1):
                segment_start = time.time()

                # Extract regime segment efficiently
                segment = data[start_idx:break_idx]
                segment_series = data_series[start_idx:break_idx]

                try:
                    # Calculate returns and positive returns once
                    returns = segment_series.pct_change().dropna()
                    positive_returns = returns > 0  # Calculate positive returns mask

                    # Efficient statistics calculation with error handling
                    regime_stats = {
                        'regime_number': regime_num,
                        'start_idx': start_idx,
                        'end_idx': break_idx,
                        'length': len(segment),
                        'mean': float(np.mean(segment)),
                        'std': float(np.std(segment)),
                        'skew': float(segment_series.skew()),
                        'kurtosis': float(segment_series.kurtosis()),
                        'min': float(np.min(segment)),
                        'max': float(np.max(segment)),
                        'median': float(np.median(segment)),
                        'volatility_5d': float(vol_5d[start_idx:break_idx].mean()),
                        'volatility_20d': float(vol_20d[start_idx:break_idx].mean())
                    }

                    # Return statistics using same returns calculation
                    return_stats = {
                        'regime_number': regime_num,
                        'mean_return': float(returns.mean()),
                        'std_return': float(returns.std()),
                        'positive_returns_pct': float(positive_returns.mean() * 100),
                        'negative_returns_pct': float((~positive_returns).mean() * 100),
                        'max_return': float(returns.max()),
                        'min_return': float(returns.min())
                    }

                    regime_data.append(regime_stats)
                    regime_returns_data.append(return_stats)

                except Exception as e:
                    print(f"Error processing regime {regime_num}: {str(e)}")
                    continue

                processing_times[f'regime_{regime_num}'] = time.time() - segment_start

                # Print progress details with additional metrics
                print(f"\nRegime {regime_num} processed:")
                print(f"- Length: {len(segment)}")
                print(f"- Mean: {regime_stats['mean']:.4f}")
                print(f"- Volatility (5d): {regime_stats['volatility_5d']:.4f}")
                print(f"- Processing time: {processing_times[f'regime_{regime_num}']:.2f} seconds")

                start_idx = break_idx
                pbar.update(1)

        # Create DataFrames with timing
        df_start_time = time.time()
        regimes_df = pd.DataFrame(regime_data)
        returns_df = pd.DataFrame(regime_returns_data)
        processing_times['dataframe_creation'] = time.time() - df_start_time

        # Enhanced wandb logging
        wandb_start_time = time.time()
        total_time = time.time() - start_time

        wandb.log({
            "regime_statistics": wandb.Table(dataframe=regimes_df),
            "regime_returns": wandb.Table(dataframe=returns_df),
            "analysis_time": total_time,
            "total_regimes": len(regimes_df),
            "processing_times": processing_times,
            "mean_regime_processing_time": np.mean([t for k, t in processing_times.items() if 'regime_' in k]),
            "regime_lengths": regimes_df['length'].tolist(),
            "mean_volatilities": regimes_df['volatility_5d'].tolist()
        })
        processing_times['wandb_logging'] = time.time() - wandb_start_time

        # Final summary
        print("\nAnalysis Complete:")
        print(f"- Total regimes processed: {len(regimes_df)}")
        print(f"- Total processing time: {total_time:.2f} seconds")
        print(f"- Average regime processing time: {np.mean([t for k, t in processing_times.items() if 'regime_' in k]):.2f} seconds")

        return regimes_df, returns_df

    except Exception as e:
        print(f"Critical error in regime analysis: {str(e)}")
        traceback.print_exc()
        raise e


# this function is called in main
def process_single_combination(args):
    """
    GPU-optimized helper function for parallel processing
    """
    start_time = time.time()
    processing_times = {}
    data, max_breaks, days = args

    # Validate and initialize
    validation_start = time.time()
    try:
        # Input validation
        if not isinstance(data, (np.ndarray, pd.Series, list)):
            raise ValueError("Input data must be a numpy array, pandas Series, or list")

        data_length = len(data)
        if data_length < 100:  # Minimum required length
            raise ValueError(f"Data length ({data_length}) is too short. Minimum required: 100")

        processing_times['validation'] = time.time() - validation_start

        # Log configuration and initial metrics
        config_data = {
            "processing_config": {
                "max_breaks": max_breaks,
                "min_size_days": days,
                "data_length": data_length,
                "start_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
        }
        wandb.log(config_data)

        print(f"\nStarting combination processing:")
        print(f"- Data length: {data_length}")
        print(f"- Max breaks: {max_breaks}")
        print(f"- Min size days: {days}")

        # GPU Processing Setup
        gpu_setup_start = time.time()
        total_steps = 5

        with tqdm(total=total_steps, desc="Processing Steps", position=0, leave=True) as main_pbar:
            # GPU Context Setup
            try:
                with cp.cuda.Device(0):  # Ensure GPU context
                    # Initialize GPU
                    gpu_init_start = time.time()
                    cp.cuda.Stream.null.synchronize()
                    initial_memory = cp.get_default_memory_pool().used_bytes()
                    processing_times['gpu_initialization'] = time.time() - gpu_init_start
                    main_pbar.update(1)
                    print(f"\nGPU Initialized. Initial memory: {initial_memory / 1e9:.2f} GB")

                    # Data Transfer to GPU
                    transfer_start = time.time()
                    main_pbar.set_description("Transferring data to GPU")
                    data_gpu = cp.asarray(data, dtype=cp.float32)  # Specify dtype for efficiency
                    min_size = convert_days_to_periods(days) / len(data)
                    processing_times['data_transfer'] = time.time() - transfer_start
                    main_pbar.update(1)
                    print(f"Data transferred to GPU. Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")

                    # Run Structural Break Analysis
                    analysis_start = time.time()
                    main_pbar.set_description("Running Structural Break Analysis")
                    break_dates, aic_scores = bai_perron_test(data_gpu, max_breaks, min_size)
                    processing_times['break_analysis'] = time.time() - analysis_start
                    main_pbar.update(1)
                    print(f"\nStructural break analysis completed:")
                    print(f"- Number of breaks found: {len(break_dates)}")
                    print(f"- Time taken: {processing_times['break_analysis']:.2f} seconds")

                    # Transfer Results to CPU
                    transfer_back_start = time.time()
                    main_pbar.set_description("Transferring results to CPU")
                    data_cpu = cp.asnumpy(data_gpu)
                    aic_scores_cpu = cp.asnumpy(aic_scores)
                    processing_times['cpu_transfer'] = time.time() - transfer_back_start
                    main_pbar.update(1)
                    print(f"Results transferred to CPU. Time taken: {processing_times['cpu_transfer']:.2f} seconds")

                    # Memory Cleanup
                    cleanup_start = time.time()
                    main_pbar.set_description("Cleaning GPU Memory")
                    initial_memory = cp.get_default_memory_pool().used_bytes()

                    # Clean GPU memory
                    mempool = cp.get_default_memory_pool()
                    pinned_mempool = cp.get_default_pinned_memory_pool()
                    mempool.free_all_blocks()
                    pinned_mempool.free_all_blocks()

                    memory_freed = initial_memory - cp.get_default_memory_pool().used_bytes()
                    processing_times['memory_cleanup'] = time.time() - cleanup_start
                    main_pbar.update(1)
                    print(f"GPU memory cleaned. Freed {memory_freed / 1e9:.2f} GB")

                    # Regime Analysis
                    analysis_start = time.time()
                    print("\nStarting regime analysis...")
                    regime_stats = analyze_regime_characteristics(data_cpu, break_dates)
                    processing_times['regime_analysis'] = time.time() - analysis_start

                    # Log final results to wandb
                    results_data = {
                        "processing_results": {
                            "max_breaks": max_breaks,
                            "min_size_days": days,
                            "n_breaks": len(break_dates),
                            "min_aic": float(np.min(aic_scores_cpu)) if len(aic_scores_cpu) > 0 else None,
                            "processing_times": processing_times,
                            "total_time": time.time() - start_time,
                            "memory_freed_gb": memory_freed / 1e9
                        }
                    }
                    wandb.log(results_data)
                    # Print execution summary
                    total_time = time.time() - start_time
                    print("\nExecution Summary:")
                    print(f"- Total processing time: {total_time:.2f} seconds")
                    for step, duration in processing_times.items():
                        print(f"- {step}: {duration:.2f} seconds")
                    print(f"- Total breaks found: {len(break_dates)}")

                    return break_dates, aic_scores_cpu, regime_stats

            except cp.cuda.runtime.CUDARuntimeError as e:
                error_time = time.time()
                print(f"\nGPU Error encountered: {str(e)}")
                print("Falling back to CPU processing...")

                # wandb.log({
                #     "gpu_error": {
                #         "message": str(e),
                #         "fallback": "CPU processing",
                #         "time_of_failure": (error_time - start_time)
                #     }
                # })

                # CPU Fallback processing
                min_size = convert_days_to_periods(days) / len(data)
                break_dates, aic_scores = bai_perron_test(data, max_breaks, min_size)
                data_cpu = data
                regime_stats = analyze_regime_characteristics(data_cpu, break_dates)

                return break_dates, aic_scores, regime_stats

    except Exception as e:
        # Comprehensive error logging
        error_info = {
            "critical_error": {
                "error_message": str(e),
                "error_type": type(e).__name__,
                "max_breaks": max_breaks,
                "min_size_days": days,
                "processing_times": processing_times,
                "total_time": time.time() - start_time
            }
        }
        # wandb.log(error_info)
        print(error_info)
        print(f"\nCritical error in processing: {str(e)}")
        traceback.print_exc()
        raise e

# this function is called in main

def load_data_efficiently(df0):
    # Convert to float32 instead of float64
    data = df0.astype(np.float32)
    # Free original dataframe memory
    del df0
    gc.collect()
    return data



## main function to run the anbalysis -
def main(df0):
    """Main execution with optimizations and progress tracking"""
    print("\n========= Starting Analysis =========")
    logger = setup_logger()
    logger.info("Starting main execution")

    start_time = time.time()
    processing_times = {}

    try:
        # Initialize parameter grid with timing
        print("\n[1/7] Initializing parameter grid...")
        grid_init_start = time.time()
        param_grid = {
            'max_breaks': [5],  # [20, 40, 60] for extended search
            'min_size_days': [5]  # [3, 5, 7] for extended search
        }
        processing_times['grid_initialization'] = time.time() - grid_init_start
        print(f"Parameter grid initialized: {param_grid}")

        # Efficient data copying
        print("\n[2/7] Loading and copying data...")
        data_copy_start = time.time()
        data = load_data_efficiently(df0)
        processing_times['data_copying'] = time.time() - data_copy_start
        print(f"Data loaded successfully. Shape: {data.shape}")
        logger.info(f"Data loaded successfully. Shape: {data.shape}")

        # Initialize wandb
        print("\n[3/7] Initializing Weights & Biases...")
        wandb_init_start = time.time()
        run = wandb.init(
            project="Time Series Analysis of Nifty50 5 min ohlc ---vhjvbhb",
            name=f"Test---regime_detection_grid_search_{datetime.now().strftime('%Y%m%d_%H%M')}",
            group="structural_breaks_analysis",
            config={
                "data_points": len(data),
                "analysis_type": "Bai_Perron_Test",
                "max_breaks_tested": param_grid['max_breaks'],
                "min_size_days_tested": param_grid['min_size_days'],
                "data_range": f"from index {0} to {len(data)-1}",
                "memory_tracking": True
            },
            tags=["structural_breaks", "regime_detection", "AIC_optimization"]
        )
        processing_times['wandb_initialization'] = time.time() - wandb_init_start
        print("Weights & Biases initialized successfully")

        # Initialize result storage
        print("\n[4/7] Preparing for parallel processing...")
        all_results = []
        all_regime_stats = []
        all_break_points = []

        combinations = [
            (data, max_breaks, days)
            for max_breaks in param_grid['max_breaks']
            for days in param_grid['min_size_days']
        ]

        total_combinations = len(combinations)
        print(f"Total parameter combinations to process: {total_combinations}")
        logger.info(f"Total parameter combinations to process: {total_combinations}")

        # Parallel processing setup
        n_cores = max(1, multiprocessing.cpu_count() - 1)
        print(f"Using {n_cores} CPU cores for parallel processing")

        # Main processing loop
        print("\n[5/7] Starting main processing loop...")
        results = []
        with tqdm(total=total_combinations, desc="Overall Progress", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as main_progress:
            with ProcessPoolExecutor(max_workers=n_cores) as executor:
                future_to_params = {
                    executor.submit(process_single_combination, combo): combo
                    for combo in combinations
                }

                for future in as_completed(future_to_params):
                    combo = future_to_params[future]
                    try:
                        break_dates, aic_scores, regime_stats = future.result()
                        results.append((break_dates, aic_scores, regime_stats))

                        # Update progress and print details
                        main_progress.update(1)
                        print(f"\nCompleted combination - Max breaks: {combo[1]}, Min size days: {combo[2]}")
                        print(f"Found {len(break_dates)} break points")
                        print(f"Progress: {len(results)}/{total_combinations}")

                        wandb.log({
                            "completed_config": {
                                "max_breaks": combo[1],
                                "min_size_days": combo[2],
                                "progress": len(results) / total_combinations
                            }
                        })

                    except Exception as e:
                        print(f"\nError in combination {combo}: {str(e)}")
                        logger.error(f"Error processing combination {combo}: {str(e)}")
                        traceback.print_exc()
                        wandb.log({
                            "processing_error": {
                                "combo": str(combo),
                                "error": str(e)
                            }
                        })

        # Process results
        print("\n[6/7] Processing final results...")
        for (max_breaks, days), (break_dates, aic_scores, regime_stats) in zip(
            [(mb, d) for mb in param_grid['max_breaks'] for d in param_grid['min_size_days']],
            results
        ):
            print(f"\nProcessing results for max_breaks={max_breaks}, min_size_days={days}")

            # Store main results
            result_entry = {
                'max_breaks': max_breaks,
                'min_size_days': days,
                'n_breaks_found': len(break_dates),
                'min_aic': float(np.min(aic_scores)),
                'mean_aic': float(np.mean(aic_scores)),
                'std_aic': float(np.std(aic_scores)),
                'n_regimes': len(regime_stats[0]),
                'timestamp': datetime.now().strftime('%Y%m%d_%H%M')
            }
            all_results.append(result_entry)
            print(f"Processed main results: {len(break_dates)} breaks found")

            # Process regime statistics
            regime_df = regime_stats[0]
            for _, row in regime_df.iterrows():
                regime_entry = {
                    'max_breaks': max_breaks,
                    'min_size_days': days,
                    'regime_number': row['regime_number'],
                    'start_idx': row['start_idx'],
                    'end_idx': row['end_idx'],
                    'length': row['length'],
                    'mean': row['mean'],
                    'std': row['std'],
                    'skew': row['skew'],
                    'kurtosis': row['kurtosis']
                }
                all_regime_stats.append(regime_entry)

            # Store break points
            for i, break_point in enumerate(break_dates):
                break_entry = {
                    'max_breaks': max_breaks,
                    'min_size_days': days,
                    'break_number': i+1,
                    'break_point_index': break_point,
                    'aic_score': aic_scores[i] if i < len(aic_scores) else None
                }
                all_break_points.append(break_entry)

        # Create final DataFrames and save results
        print("\n[7/7] Saving final results...")
        results_df = pd.DataFrame(all_results)
        regimes_df = pd.DataFrame(all_regime_stats)
        breaks_df = pd.DataFrame(all_break_points)

        # # Save to CSV
        # results_df.to_csv('structural_breaks_summary.csv', index=False)
        # regimes_df.to_csv('regime_statistics.csv', index=False)
        # breaks_df.to_csv('break_points.csv', index=False)
        # print("Results saved to CSV files")

        # Final Wandb logging
        wandb.log({
            "final_summary_table": wandb.Table(dataframe=results_df),
            "final_regime_statistics": wandb.Table(dataframe=regimes_df),
            "final_break_points": wandb.Table(dataframe=breaks_df),
            "completed_configurations": len(results_df),
            "total_processing_time": time.time() - start_time
        })

        total_time = time.time() - start_time
        print(f"\n========= Analysis Complete =========")
        print(f"Total processing time: {total_time:.2f} seconds")
        print(f"Results saved in: structural_breaks_summary.csv")
        print(f"Regime statistics saved in: regime_statistics.csv")
        print(f"Break points saved in: break_points.csv")

        logger.info("Analysis completed successfully")
        wandb.finish()

        return results_df, regimes_df, breaks_df

    except Exception as e:
        print(f"\n========= ERROR =========")
        print(f"Critical error in main execution: {str(e)}")
        logger.error(f"Critical error in main execution: {str(e)}")
        traceback.print_exc()
        if wandb.run is not None:
            wandb.log({"critical_error": str(e)})
            wandb.finish()
        raise e


if __name__ == "__main__":
    return_series = returns_df["returns"]
    df0 = load_data_efficiently(return_series)
    results_df, regimes_df, breaks_df = main(return_series)




INFO:MyLogger:Starting main execution
INFO:MyLogger:Data loaded successfully. Shape: (178117,)
INFO:MyLogger:Total parameter combinations to process: 1



========= Starting Analysis =========

[1/7] Initializing parameter grid...
Parameter grid initialized: {'max_breaks': [5], 'min_size_days': [5]}

[2/7] Loading and copying data...
Data loaded successfully. Shape: (178117,)

[3/7] Initializing Weights & Biases...
Weights & Biases initialized successfully

[4/7] Preparing for parallel processing...
Total parameter combinations to process: 1
Using 11 CPU cores for parallel processing

[5/7] Starting main processing loop...


Overall Progress:   0%|          | 0/1 [00:00<?]


Starting combination processing:
- Data length: 178117
- Max breaks: 5
- Min size days: 5


Processing Steps:   0%|          | 0/5 [00:00<?, ?it/s]

...


Memory cleared at index 8400 - Current usage: 0.01 GB

Progress - Index 8400/10375:
Segment sizes - Left: 8400, Right: 169717
Memory: 0.01 GB

Memory cleared at index 8700 - Current usage: 0.01 GB

Progress - Index 8700/10375:
Segment sizes - Left: 8700, Right: 169417
Memory: 0.01 GB

Memory cleared at index 9000 - Current usage: 0.01 GB

Progress - Index 9000/10375:
Segment sizes - Left: 9000, Right: 169117
Memory: 0.01 GB

Memory cleared at index 9300 - Current usage: 0.01 GB

Progress - Index 9300/10375:
Segment sizes - Left: 9300, Right: 168817
Memory: 0.01 GB

Memory cleared at index 9600 - Current usage: 0.01 GB

Progress - Index 9600/10375:
Segment sizes - Left: 9600, Right: 168517
Memory: 0.01 GB

Memory cleared at index 9900 - Current usage: 0.01 GB

Progress - Index 9900/10375:
Segment sizes - Left: 9900, Right: 168217
Memory: 0.01 GB

Memory cleared at index 10200 - Current usage: 0.01 GB

Progress - Index 10200/10375:
Segment sizes - Left: 10200, Right: 167917
Memory: 0.01

Processing indices 10375-12375:   0%|          | 0/2000 [00:00<?] 


Memory cleared at index 10500 - Current usage: 0.01 GB

Progress - Index 10500/12375:
Segment sizes - Left: 10500, Right: 167617
Memory: 0.01 GB

Memory cleared at index 10800 - Current usage: 0.01 GB

Progress - Index 10800/12375:
Segment sizes - Left: 10800, Right: 167317
Memory: 0.01 GB

Memory cleared at index 11100 - Current usage: 0.01 GB

Progress - Index 11100/12375:
Segment sizes - Left: 11100, Right: 167017
Memory: 0.01 GB

Memory cleared at index 11400 - Current usage: 0.01 GB

Progress - Index 11400/12375:
Segment sizes - Left: 11400, Right: 166717
Memory: 0.01 GB

Memory cleared at index 11700 - Current usage: 0.01 GB

Progress - Index 11700/12375:
Segment sizes - Left: 11700, Right: 166417
Memory: 0.01 GB

Memory cleared at index 12000 - Current usage: 0.01 GB

Progress - Index 12000/12375:
Segment sizes - Left: 12000, Right: 166117
Memory: 0.01 GB

Memory cleared at index 12300 - Current usage: 0.01 GB

Progress - Index 12300/12375:
Segment sizes - Left: 12300, Right: 1

Processing indices 12375-14375:   0%|          | 0/2000 [00:00<?] 


Memory cleared at index 12600 - Current usage: 0.01 GB

Progress - Index 12600/14375:
Segment sizes - Left: 12600, Right: 165517
Memory: 0.01 GB

Memory cleared at index 12900 - Current usage: 0.01 GB

Progress - Index 12900/14375:
Segment sizes - Left: 12900, Right: 165217
Memory: 0.01 GB

Memory cleared at index 13200 - Current usage: 0.01 GB

Progress - Index 13200/14375:
Segment sizes - Left: 13200, Right: 164917
Memory: 0.01 GB


In [ ]:
return_series = returns_df["returns"]
return_series

,returns
date,
2015-01-09 09:20:00+05:30,-0.000024
2015-01-09 09:25:00+05:30,-0.000825
2015-01-09 09:30:00+05:30,-0.000681
2015-01-09 09:35:00+05:30,-0.000609
2015-01-09 09:40:00+05:30,0.000254
...,...
2016-10-04 15:20:00+05:30,-0.000256
2016-10-04 15:25:00+05:30,-0.000416
2016-10-05 09:15:00+05:30,0.002286


In [ ]:
####### Libraries -
# @title
import pandas as pd
import numpy as np
import wandb
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
from functools import lru_cache
from numba import jit
from tqdm.auto import tqdm
import time
import cupy as cp
from concurrent.futures import ProcessPoolExecutor, as_completed
import logging
import traceback

In [ ]:
# Configure the logger
def setup_logger():
    # Create a logger
    logger = logging.getLogger("MyLogger")
    logger.setLevel(logging.DEBUG)  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)

    # Create a file handler
    file_handler = logging.FileHandler("application.log")  # Specify the log file name
    file_handler.setLevel(logging.DEBUG)  # Set the logging level for the handler

    # Create a formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)  # Attach the formatter to the handler

    # Add the handler to the logger
    logger.addHandler(file_handler)

    return logger

In [ ]:
#### this function is being used in plot regimes and process single combination
@jit(nopython=True)
def convert_days_to_periods(days, timeframe=5):
    """Numba optimized version"""
    minutes_per_day = 375  # Trading hours
    periods_per_day = minutes_per_day // timeframe
    return days * periods_per_day

### this function is used in bai perron test
@jit(nopython=True)
def calculate_aic(rss, n_params, n_samples):
    """Numba optimized AIC calculation"""
    return n_samples * np.log(rss/n_samples) + 2 * n_params

### this function is used in bai perron test
@lru_cache(maxsize=1000)
def cached_ols_fit(data_key, X_key):
    """
    Cached OLS fitting with GPU support and error handling

    Parameters:
    -----------
    data_key : tuple
        Data array converted to tuple for caching
    X_key : tuple
        Design matrix converted to tuple for caching

    Returns:
    --------
    OLS fitted model
    """
    try:
        # Convert to numpy arrays
        data = np.array(data_key)
        X = np.array(X_key)

        # Check dimensions
        if X.shape[0] != data.shape[0]:
            raise ValueError("Data and design matrix dimensions mismatch")

        # Fit OLS model
        model = OLS(data, X).fit()

        return model

    except Exception as e:
        print(f"Error in OLS fitting: {e}")
        raise e

In [ ]:
### this function is used in process single combination
def bai_perron_test(data, max_breaks=5, min_size=0.1, early_stopping_patience=3):
    start_time = time.time()
    processing_times = {}
    try:
        # Input validation with timing
        validation_start = time.time()
        if len(data) < 100:
            raise ValueError("Data length must be at least 100 points")
        if max_breaks < 1:
            raise ValueError("max_breaks must be at least 1")
        if not (0 < min_size < 0.5):
            raise ValueError("min_size must be between 0 and 0.5")
        processing_times['validation'] = time.time() - validation_start

        print(f"Starting analysis with data length: {len(data)}")
        print(f"Parameters: max_breaks={max_breaks}, min_size={min_size}")

        # Data prep
        data_prep_start = time.time()
        data_gpu = cp.asarray(data, dtype=cp.float32)
        n = len(data_gpu)
        min_size_obs = int(min_size * n)
        processing_times['data_preparation'] = time.time() - data_prep_start
        print(f"Data preparation completed. Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")

        # Optimized Array Initialization with timing
        array_init_start = time.time()
        rss_array = cp.zeros((n, max_breaks+1), dtype=cp.float32)
        break_points = cp.zeros((n, max_breaks+1), dtype=cp.int32)
        aic_scores = cp.zeros(max_breaks+1, dtype=cp.float32)
        processing_times['array_initialization'] = time.time() - array_init_start

        # Base case calculation with timing
        base_calc_start = time.time()
        print("\nStarting base case calculations...")

        X_gpu = cp.column_stack((
            cp.ones(n, dtype=cp.float32),
            cp.arange(n, dtype=cp.float32)
        ))

        # Calculate base case (no breaks)

        beta_base = cp.linalg.solve(X_gpu.T @ X_gpu, X_gpu.T @ data_gpu)
        residuals = data_gpu - X_gpu @ beta_base
        base_rss = float(cp.sum(residuals**2))
        rss_array[0, 0] = base_rss
        aic_scores[0] = calculate_aic(base_rss, 2, n)

        processing_times['base_case'] = time.time() - base_calc_start
        print(f"Base case completed:")
        print(f"- Base RSS: {base_rss:.4e}")
        print(f"- Initial AIC: {float(aic_scores[0]):.4e}")
        print(f"- Time taken: {processing_times['base_case']:.2f} seconds")
        print(f"- Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")

        # Batch Processing for Large Datasets
        batch_start_time = time.time()
        batch_size = min(5000, n//20)
        no_improve = 0
        best_aic = float('inf')
        total_batches = ((n-min_size_obs+1) - min_size_obs) // batch_size + 1

        print(f"\nStarting batch processing:")
        print(f"- Batch size: {batch_size}")
        print(f"- Total batches per break: {total_batches}")

        with tqdm(total=max_breaks, desc="Break Detection", position=0, leave=True) as pbar:
            for m in range(1, max_breaks+1):
                break_start_time = time.time()
                min_aic = float('inf')
                best_break = None
                batch_count = 0

                # Process in batches with nested progress bar
                with tqdm(total=total_batches, desc=f"Break {m}", position=1, leave=False) as batch_pbar:
                    for start_idx in range(min_size_obs, n-min_size_obs+1, batch_size):
                        batch_count += 1
                        end_idx = min(start_idx + batch_size, n-min_size_obs+1)

                        # Memory management
                        if start_idx % (batch_size * 5) == 0:
                            cp.cuda.Stream.null.synchronize()
                            mempool = cp.get_default_memory_pool()
                            mempool.free_all_blocks()
                            print(f"\nMemory cleared - Current usage: {mempool.used_bytes() / 1e9:.2f} GB")

                        # Process batch
                        for i in range(start_idx, end_idx):
                            y1, y2 = data_gpu[:i], data_gpu[i:]
                            X1, X2 = X_gpu[:i], X_gpu[i:]

                            # Convert to tuples for caching
                            y1_tuple = tuple(cp.asnumpy(y1))
                            y2_tuple = tuple(cp.asnumpy(y2))
                            X1_tuple = tuple(map(tuple, cp.asnumpy(X1)))
                            X2_tuple = tuple(map(tuple, cp.asnumpy(X2)))

                            try:
                                # Use cached OLS fitting
                                model1 = cached_ols_fit(y1_tuple, X1_tuple)
                                model2 = cached_ols_fit(y2_tuple, X2_tuple)

                                # Get predictions
                                residuals1 = y1 - cp.asarray(model1.predict(cp.asnumpy(X1)))
                                residuals2 = y2 - cp.asarray(model2.predict(cp.asnumpy(X2)))

                                rss = float(cp.sum(residuals1**2) + cp.sum(residuals2**2))
                                current_aic = calculate_aic(rss, 2*(m+1), n)

                                if current_aic < min_aic:
                                    min_aic = current_aic
                                    best_break = i

                            except Exception:
                                continue  # Skip if any error occurs in fitting

                        batch_pbar.update(1)
                        batch_pbar.set_postfix({
                            'Best AIC': f'{min_aic:.2f}',
                            'Best Break': best_break
                        })

                # Early stopping with accuracy preservation
                if min_aic > best_aic:
                    no_improve += 1
                    if no_improve >= early_stopping_patience:
                        print("\nEarly stopping triggered")
                        break
                else:
                    best_aic = min_aic
                    no_improve = 0

                if best_break is not None:
                    break_points[:, m] = best_break
                    aic_scores[m] = min_aic

                break_time = time.time() - break_start_time
                print(f"\nBreak {m} completed:")
                print(f"- Time taken: {break_time:.2f} seconds")
                print(f"- Best AIC: {min_aic:.2f}")
                print(f"- Best break point: {best_break}")

                pbar.update(1)
                pbar.set_postfix({'AIC': f'{min_aic:.2f}'})

        processing_times['batch_processing'] = time.time() - batch_start_time

        # Results Processing and Memory Cleanup
        results_start_time = time.time()
        print("\nProcessing final results...")

        # Find optimal breaks
        optimal_breaks = int(cp.argmin(aic_scores))
        break_dates = sorted([
            int(break_points[i, i+1])
            for i in range(optimal_breaks)
        ])

        print(f"\nResults summary:")
        print(f"- Optimal number of breaks: {optimal_breaks}")
        print(f"- Break points: {break_dates}")
        print(f"- Final AIC score: {float(aic_scores[optimal_breaks]):.4e}")

        # Memory cleanup
        print("\nCleaning up GPU memory...")
        mempool = cp.get_default_memory_pool()
        pinned_mempool = cp.get_default_pinned_memory_pool()
        initial_memory = mempool.used_bytes()

        mempool.free_all_blocks()
        pinned_mempool.free_all_blocks()

        final_memory = mempool.used_bytes()
        memory_freed = initial_memory - final_memory

        print(f"- Memory freed: {memory_freed / 1e9:.2f} GB")

        # Timing summary
        processing_times['results_processing'] = time.time() - results_start_time
        total_time = time.time() - start_time

        print("\nTiming Summary:")
        for step, duration in processing_times.items():
            print(f"- {step}: {duration:.2f} seconds")
        print(f"- Total execution time: {total_time:.2f} seconds")

        return break_dates, cp.asnumpy(aic_scores)

    except Exception as e:
        print(f"Error in bai_perron_test: {str(e)}")
        traceback.print_exc()
        raise e

In [ ]:
## this function is used in process single combination
def analyze_regime_characteristics(data, break_dates):
    processing_times = {}
    start_time = time.time()

    # Initialize data storage lists
    regime_data = []
    regime_returns_data = []

    # Validate inputs
    validation_start = time.time()
    try:
        if not isinstance(break_dates, (list, np.ndarray)):
            raise ValueError("break_dates must be a list or numpy array")
        if len(break_dates) == 0:
            print("Warning: No break dates provided, treating entire series as one regime")
            break_dates = []
    except Exception as e:
        print(f"Error in input validation: {e}")
        raise
    processing_times['validation'] = time.time() - validation_start

    print(f"\nStarting regime analysis:")
    print(f"- Number of breaks: {len(break_dates)}")
    print(f"- Data length: {len(data)}")

    # Initialize timing and calculation helpers
    calc_start_time = time.time()
    data_series = pd.Series(data)  # Convert once instead of multiple times

    # Cache rolling calculations
    vol_5d = data_series.rolling(5).std()
    vol_20d = data_series.rolling(20).std()

    processing_times['preparation'] = time.time() - calc_start_time
    print(f"Data preparation completed in {processing_times['preparation']:.2f} seconds")

    try:
        with tqdm(total=len(break_dates) + 1, desc="Analyzing Regimes") as pbar:
            start_idx = 0

            for regime_num, break_idx in enumerate(break_dates + [len(data)], 1):
                segment_start = time.time()

                # Extract regime segment efficiently
                segment = data[start_idx:break_idx]
                segment_series = data_series[start_idx:break_idx]

                try:
                    # Calculate returns and positive returns once
                    returns = segment_series.pct_change().dropna()
                    positive_returns = returns > 0  # Calculate positive returns mask

                    # Efficient statistics calculation with error handling
                    regime_stats = {
                        'regime_number': regime_num,
                        'start_idx': start_idx,
                        'end_idx': break_idx,
                        'length': len(segment),
                        'mean': float(np.mean(segment)),
                        'std': float(np.std(segment)),
                        'skew': float(segment_series.skew()),
                        'kurtosis': float(segment_series.kurtosis()),
                        'min': float(np.min(segment)),
                        'max': float(np.max(segment)),
                        'median': float(np.median(segment)),
                        'volatility_5d': float(vol_5d[start_idx:break_idx].mean()),
                        'volatility_20d': float(vol_20d[start_idx:break_idx].mean())
                    }

                    # Return statistics using same returns calculation
                    return_stats = {
                        'regime_number': regime_num,
                        'mean_return': float(returns.mean()),
                        'std_return': float(returns.std()),
                        'positive_returns_pct': float(positive_returns.mean() * 100),
                        'negative_returns_pct': float((~positive_returns).mean() * 100),
                        'max_return': float(returns.max()),
                        'min_return': float(returns.min())
                    }

                    regime_data.append(regime_stats)
                    regime_returns_data.append(return_stats)

                except Exception as e:
                    print(f"Error processing regime {regime_num}: {str(e)}")
                    continue

                processing_times[f'regime_{regime_num}'] = time.time() - segment_start

                # Print progress details with additional metrics
                print(f"\nRegime {regime_num} processed:")
                print(f"- Length: {len(segment)}")
                print(f"- Mean: {regime_stats['mean']:.4f}")
                print(f"- Volatility (5d): {regime_stats['volatility_5d']:.4f}")
                print(f"- Processing time: {processing_times[f'regime_{regime_num}']:.2f} seconds")

                start_idx = break_idx
                pbar.update(1)

        # Create DataFrames with timing
        df_start_time = time.time()
        regimes_df = pd.DataFrame(regime_data)
        returns_df = pd.DataFrame(regime_returns_data)
        processing_times['dataframe_creation'] = time.time() - df_start_time

        # Enhanced wandb logging
        wandb_start_time = time.time()
        total_time = time.time() - start_time

        wandb.log({
            "regime_statistics": wandb.Table(dataframe=regimes_df),
            "regime_returns": wandb.Table(dataframe=returns_df),
            "analysis_time": total_time,
            "total_regimes": len(regimes_df),
            "processing_times": processing_times,
            "mean_regime_processing_time": np.mean([t for k, t in processing_times.items() if 'regime_' in k]),
            "regime_lengths": regimes_df['length'].tolist(),
            "mean_volatilities": regimes_df['volatility_5d'].tolist()
        })
        processing_times['wandb_logging'] = time.time() - wandb_start_time

        # Final summary
        print("\nAnalysis Complete:")
        print(f"- Total regimes processed: {len(regimes_df)}")
        print(f"- Total processing time: {total_time:.2f} seconds")
        print(f"- Average regime processing time: {np.mean([t for k, t in processing_times.items() if 'regime_' in k]):.2f} seconds")

        return regimes_df, returns_df

    except Exception as e:
        print(f"Critical error in regime analysis: {str(e)}")
        traceback.print_exc()
        raise e


In [ ]:
## this function is used in process single combination
def plot_regimes(data, break_dates, regime_stats):
    """
    Plot time series regimes with statistical annotations and volatility.

    Parameters:
    -----------
    data : array-like
        Time series data
    break_dates : list
        Indices of structural break points
    regime_stats : tuple or list
        Regime statistical information

    Returns:
    --------
    fig : matplotlib.figure.Figure
        Generated visualization
    """
    logger = setup_logger()

    try:
        # Validate inputs
        if not isinstance(data, (np.ndarray, pd.Series, list)):
            raise ValueError("Data must be a numpy array, pandas Series, or list")

        # Replace logger warnings with print statements or wandb logging
        if not break_dates:
            print("Warning: No break dates provided. Plotting entire series as single regime.")
            break_dates = []

        if not regime_stats:
            print("Warning: No regime statistics provided. Creating default statistics.")
            regime_stats = [{'mean': np.mean(data), 'std': np.std(data)} for _ in range(len(break_dates) + 1)]

        # Unpack regime statistics
        if isinstance(regime_stats, tuple):
            regimes_df, returns_df = regime_stats
            regime_stats = regimes_df.to_dict('records')


        # Create figure
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), height_ratios=[3, 1])

        # Generate colors for regimes
        colors = plt.cm.rainbow(np.linspace(0, 1, len(regime_stats)))

        # Plot each regime with different color
        start_idx = 0
        for i, (end_idx, color) in enumerate(zip(break_dates + [len(data)], colors)):
            try:
                regime_data = data[start_idx:end_idx]
                regime_index = range(start_idx, end_idx)

                # Main price plot with regime coloring
                ax1.plot(regime_index, regime_data, color=color, label=f'Regime {i+1}')

                # Add regime statistics as text
                stats = regime_stats[i]
                mid_point = start_idx + len(regime_data)//2

                # Safely access mean and std
                mean = stats.get('mean', stats.get('Mean', 0))
                std = stats.get('std', stats.get('Std', 0))

                ax1.text(mid_point, ax1.get_ylim()[1],
                         f'R{i+1}\nμ={mean:.2e}\nσ={std:.2e}',
                         horizontalalignment='center', alpha=0.7)

                # Volatility subplot with safety check
                rolling_window = min(75, max(3, len(regime_data) // 2))
                volatility = (pd.Series(regime_data).rolling(rolling_window, min_periods=1).std()
                              if len(regime_data) > rolling_window
                              else pd.Series(np.std(regime_data) * np.ones(len(regime_data))))

                ax2.plot(regime_index, volatility, color=color, alpha=0.7)

                start_idx = end_idx

            except Exception as regime_error:
                logger.error(f"Error processing regime {i+1}: {regime_error}")
                continue

        # Add vertical lines at break points
        for date in break_dates:
            ax1.axvline(x=date, color='black', linestyle='--', alpha=0.3)
            ax2.axvline(x=date, color='black', linestyle='--', alpha=0.3)

        # Styling and labels
        ax1.set_title('Time Series with Regime Breaks and Statistics')
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax1.set_ylabel('Price')

        ax2.set_title('Rolling Volatility by Regime')
        ax2.set_xlabel('Time Index')
        ax2.set_ylabel('Volatility')

        plt.tight_layout()

        # Log plot to wandb
        try:
            wandb.log({"detailed_regime_plot": wandb.Image(fig)})
        except Exception as wandb_error:
            logger.warning(f"Could not log plot to wandb: {wandb_error}")

        plt.close()

        return fig

    except Exception as main_error:
        logger.error(f"Critical error in plot_regimes: {main_error}")
        return None

In [ ]:
# this function is called in main
def process_single_combination(args):
    """
    GPU-optimized helper function for parallel processing
    """
    start_time = time.time()
    processing_times = {}
    data, max_breaks, days = args

    # Validate and initialize
    validation_start = time.time()
    try:
        # Input validation
        if not isinstance(data, (np.ndarray, pd.Series, list)):
            raise ValueError("Input data must be a numpy array, pandas Series, or list")

        data_length = len(data)
        if data_length < 100:  # Minimum required length
            raise ValueError(f"Data length ({data_length}) is too short. Minimum required: 100")

        processing_times['validation'] = time.time() - validation_start

        # Log configuration and initial metrics
        config_data = {
            "processing_config": {
                "max_breaks": max_breaks,
                "min_size_days": days,
                "data_length": data_length,
                "start_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
        }
        wandb.log(config_data)

        print(f"\nStarting combination processing:")
        print(f"- Data length: {data_length}")
        print(f"- Max breaks: {max_breaks}")
        print(f"- Min size days: {days}")

        # GPU Processing Setup
        gpu_setup_start = time.time()
        total_steps = 5

        with tqdm(total=total_steps, desc="Processing Steps", position=0, leave=True) as main_pbar:
            # GPU Context Setup
            try:
                with cp.cuda.Device(0):  # Ensure GPU context
                    # Initialize GPU
                    gpu_init_start = time.time()
                    cp.cuda.Stream.null.synchronize()
                    initial_memory = cp.get_default_memory_pool().used_bytes()
                    processing_times['gpu_initialization'] = time.time() - gpu_init_start
                    main_pbar.update(1)
                    print(f"\nGPU Initialized. Initial memory: {initial_memory / 1e9:.2f} GB")

                    # Data Transfer to GPU
                    transfer_start = time.time()
                    main_pbar.set_description("Transferring data to GPU")
                    data_gpu = cp.asarray(data, dtype=cp.float32)  # Specify dtype for efficiency
                    min_size = convert_days_to_periods(days) / len(data)
                    processing_times['data_transfer'] = time.time() - transfer_start
                    main_pbar.update(1)
                    print(f"Data transferred to GPU. Memory usage: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")

                    # Run Structural Break Analysis
                    analysis_start = time.time()
                    main_pbar.set_description("Running Structural Break Analysis")
                    break_dates, aic_scores = bai_perron_test(data_gpu, max_breaks, min_size)
                    processing_times['break_analysis'] = time.time() - analysis_start
                    main_pbar.update(1)
                    print(f"\nStructural break analysis completed:")
                    print(f"- Number of breaks found: {len(break_dates)}")
                    print(f"- Time taken: {processing_times['break_analysis']:.2f} seconds")

                    # Transfer Results to CPU
                    transfer_back_start = time.time()
                    main_pbar.set_description("Transferring results to CPU")
                    data_cpu = cp.asnumpy(data_gpu)
                    aic_scores_cpu = cp.asnumpy(aic_scores)
                    processing_times['cpu_transfer'] = time.time() - transfer_back_start
                    main_pbar.update(1)
                    print(f"Results transferred to CPU. Time taken: {processing_times['cpu_transfer']:.2f} seconds")

                    # Memory Cleanup
                    cleanup_start = time.time()
                    main_pbar.set_description("Cleaning GPU Memory")
                    initial_memory = cp.get_default_memory_pool().used_bytes()

                    # Clean GPU memory
                    mempool = cp.get_default_memory_pool()
                    pinned_mempool = cp.get_default_pinned_memory_pool()
                    mempool.free_all_blocks()
                    pinned_mempool.free_all_blocks()

                    memory_freed = initial_memory - cp.get_default_memory_pool().used_bytes()
                    processing_times['memory_cleanup'] = time.time() - cleanup_start
                    main_pbar.update(1)
                    print(f"GPU memory cleaned. Freed {memory_freed / 1e9:.2f} GB")

                    # Regime Analysis
                    analysis_start = time.time()
                    print("\nStarting regime analysis...")
                    regime_stats = analyze_regime_characteristics(data_cpu, break_dates)
                    processing_times['regime_analysis'] = time.time() - analysis_start

                    # Log final results to wandb
                    results_data = {
                        "processing_results": {
                            "max_breaks": max_breaks,
                            "min_size_days": days,
                            "n_breaks": len(break_dates),
                            "min_aic": float(np.min(aic_scores_cpu)) if len(aic_scores_cpu) > 0 else None,
                            "processing_times": processing_times,
                            "total_time": time.time() - start_time,
                            "memory_freed_gb": memory_freed / 1e9
                        }
                    }
                    wandb.log(results_data)
                    # Print execution summary
                    total_time = time.time() - start_time
                    print("\nExecution Summary:")
                    print(f"- Total processing time: {total_time:.2f} seconds")
                    for step, duration in processing_times.items():
                        print(f"- {step}: {duration:.2f} seconds")
                    print(f"- Total breaks found: {len(break_dates)}")

                    return break_dates, aic_scores_cpu, regime_stats

            except cp.cuda.runtime.CUDARuntimeError as e:
                error_time = time.time()
                print(f"\nGPU Error encountered: {str(e)}")
                print("Falling back to CPU processing...")

                # wandb.log({
                #     "gpu_error": {
                #         "message": str(e),
                #         "fallback": "CPU processing",
                #         "time_of_failure": (error_time - start_time)
                #     }
                # })

                # CPU Fallback processing
                min_size = convert_days_to_periods(days) / len(data)
                break_dates, aic_scores = bai_perron_test(data, max_breaks, min_size)
                data_cpu = data
                regime_stats = analyze_regime_characteristics(data_cpu, break_dates)

                return break_dates, aic_scores, regime_stats

    except Exception as e:
        # Comprehensive error logging
        error_info = {
            "critical_error": {
                "error_message": str(e),
                "error_type": type(e).__name__,
                "max_breaks": max_breaks,
                "min_size_days": days,
                "processing_times": processing_times,
                "total_time": time.time() - start_time
            }
        }
        # wandb.log(error_info)
        print(error_info)
        print(f"\nCritical error in processing: {str(e)}")
        traceback.print_exc()
        raise e

In [ ]:
def main(df0):
    """Main execution with optimizations and progress tracking"""
    print("\n========= Starting Analysis =========")
    logger = setup_logger()
    logger.info("Starting main execution")

    start_time = time.time()
    processing_times = {}

    try:
        # Initialize parameter grid with timing
        print("\n[1/7] Initializing parameter grid...")
        grid_init_start = time.time()
        param_grid = {
            'max_breaks': [20],  # [20, 40, 60] for extended search
            'min_size_days': [5]  # [3, 5, 7] for extended search
        }
        processing_times['grid_initialization'] = time.time() - grid_init_start
        print(f"Parameter grid initialized: {param_grid}")

        # Efficient data copying
        print("\n[2/7] Loading and copying data...")
        data_copy_start = time.time()
        data = load_data_efficiently(df0)
        processing_times['data_copying'] = time.time() - data_copy_start
        print(f"Data loaded successfully. Shape: {data.shape}")
        logger.info(f"Data loaded successfully. Shape: {data.shape}")

        # Initialize wandb
        print("\n[3/7] Initializing Weights & Biases...")
        wandb_init_start = time.time()
        run = wandb.init(
            project="Time Series Analysis of Nifty50 5 min ohlc",
            name=f"Test---regime_detection_grid_search_{datetime.now().strftime('%Y%m%d_%H%M')}",
            group="structural_breaks_analysis",
            config={
                "data_points": len(data),
                "analysis_type": "Bai_Perron_Test",
                "max_breaks_tested": param_grid['max_breaks'],
                "min_size_days_tested": param_grid['min_size_days'],
                "data_range": f"from index {0} to {len(data)-1}",
                "memory_tracking": True
            },
            tags=["structural_breaks", "regime_detection", "AIC_optimization"]
        )
        processing_times['wandb_initialization'] = time.time() - wandb_init_start
        print("Weights & Biases initialized successfully")

        # Initialize result storage
        print("\n[4/7] Preparing for parallel processing...")
        all_results = []
        all_regime_stats = []
        all_break_points = []

        combinations = [
            (data, max_breaks, days)
            for max_breaks in param_grid['max_breaks']
            for days in param_grid['min_size_days']
        ]

        total_combinations = len(combinations)
        print(f"Total parameter combinations to process: {total_combinations}")
        logger.info(f"Total parameter combinations to process: {total_combinations}")

        # Parallel processing setup
        n_cores = max(1, multiprocessing.cpu_count() - 1)
        print(f"Using {n_cores} CPU cores for parallel processing")

        # Main processing loop
        print("\n[5/7] Starting main processing loop...")
        results = []
        with tqdm(total=total_combinations, desc="Overall Progress", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as main_progress:
            with ProcessPoolExecutor(max_workers=n_cores) as executor:
                future_to_params = {
                    executor.submit(process_single_combination, combo): combo
                    for combo in combinations
                }

                for future in as_completed(future_to_params):
                    combo = future_to_params[future]
                    try:
                        break_dates, aic_scores, regime_stats = future.result()
                        results.append((break_dates, aic_scores, regime_stats))

                        # Update progress and print details
                        main_progress.update(1)
                        print(f"\nCompleted combination - Max breaks: {combo[1]}, Min size days: {combo[2]}")
                        print(f"Found {len(break_dates)} break points")
                        print(f"Progress: {len(results)}/{total_combinations}")

                        wandb.log({
                            "completed_config": {
                                "max_breaks": combo[1],
                                "min_size_days": combo[2],
                                "progress": len(results) / total_combinations
                            }
                        })

                    except Exception as e:
                        print(f"\nError in combination {combo}: {str(e)}")
                        logger.error(f"Error processing combination {combo}: {str(e)}")
                        traceback.print_exc()
                        wandb.log({
                            "processing_error": {
                                "combo": str(combo),
                                "error": str(e)
                            }
                        })

        # Process results
        print("\n[6/7] Processing final results...")
        for (max_breaks, days), (break_dates, aic_scores, regime_stats) in zip(
            [(mb, d) for mb in param_grid['max_breaks'] for d in param_grid['min_size_days']],
            results
        ):
            print(f"\nProcessing results for max_breaks={max_breaks}, min_size_days={days}")

            # Store main results
            result_entry = {
                'max_breaks': max_breaks,
                'min_size_days': days,
                'n_breaks_found': len(break_dates),
                'min_aic': float(np.min(aic_scores)),
                'mean_aic': float(np.mean(aic_scores)),
                'std_aic': float(np.std(aic_scores)),
                'n_regimes': len(regime_stats[0]),
                'timestamp': datetime.now().strftime('%Y%m%d_%H%M')
            }
            all_results.append(result_entry)
            print(f"Processed main results: {len(break_dates)} breaks found")

            # Process regime statistics
            regime_df = regime_stats[0]
            for _, row in regime_df.iterrows():
                regime_entry = {
                    'max_breaks': max_breaks,
                    'min_size_days': days,
                    'regime_number': row['regime_number'],
                    'start_idx': row['start_idx'],
                    'end_idx': row['end_idx'],
                    'length': row['length'],
                    'mean': row['mean'],
                    'std': row['std'],
                    'skew': row['skew'],
                    'kurtosis': row['kurtosis']
                }
                all_regime_stats.append(regime_entry)

            # Store break points
            for i, break_point in enumerate(break_dates):
                break_entry = {
                    'max_breaks': max_breaks,
                    'min_size_days': days,
                    'break_number': i+1,
                    'break_point_index': break_point,
                    'aic_score': aic_scores[i] if i < len(aic_scores) else None
                }
                all_break_points.append(break_entry)

        # Create final DataFrames and save results
        print("\n[7/7] Saving final results...")
        results_df = pd.DataFrame(all_results)
        regimes_df = pd.DataFrame(all_regime_stats)
        breaks_df = pd.DataFrame(all_break_points)

        # # Save to CSV
        # results_df.to_csv('structural_breaks_summary.csv', index=False)
        # regimes_df.to_csv('regime_statistics.csv', index=False)
        # breaks_df.to_csv('break_points.csv', index=False)
        # print("Results saved to CSV files")

        # Final Wandb logging
        wandb.log({
            "final_summary_table": wandb.Table(dataframe=results_df),
            "final_regime_statistics": wandb.Table(dataframe=regimes_df),
            "final_break_points": wandb.Table(dataframe=breaks_df),
            "completed_configurations": len(results_df),
            "total_processing_time": time.time() - start_time
        })

        total_time = time.time() - start_time
        print(f"\n========= Analysis Complete =========")
        print(f"Total processing time: {total_time:.2f} seconds")
        print(f"Results saved in: structural_breaks_summary.csv")
        print(f"Regime statistics saved in: regime_statistics.csv")
        print(f"Break points saved in: break_points.csv")

        logger.info("Analysis completed successfully")
        wandb.finish()

        return results_df, regimes_df, breaks_df

    except Exception as e:
        print(f"\n========= ERROR =========")
        print(f"Critical error in main execution: {str(e)}")
        logger.error(f"Critical error in main execution: {str(e)}")
        traceback.print_exc()
        if wandb.run is not None:
            wandb.log({"critical_error": str(e)})
            wandb.finish()
        raise e

def load_data_efficiently(df0):
    # Convert to float32 instead of float64
    data = df0.astype(np.float32)
    # Free original dataframe memory
    del df0
    gc.collect()
    return data


if __name__ == "__main__":
    df0 = load_data_efficiently(returns_df)
    results_df, regimes_df, breaks_df = main(df0)

In [ ]:
# @title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf
import wandb
from typing import Dict, List, Tuple
from datetime import datetime

class TimeSeriesAnalysis:
    ACF_KEY: str = 'acf_values'
    PACF_KEY: str = 'pacf_values'
    ACF_LAGS_KEY: str = 'acf_lags'
    PACF_LAGS_KEY: str = 'pacf_lags'

    def __init__(self, returns_series, max_lags=200):
        self.returns = returns_series
        self.max_lags = max_lags

        wandb.init(
            project="Time Series Analysis of Nifty50 5 min ohlc",
            name=f"ACF_PACF_last_half_data_{datetime.now().strftime('%Y%m%d_%H%M')}",
            group="ACF_PACF_analysis",
            config={
                "data_points": len(returns_series),
                "analysis_type": "ACF_PACF",
                "data_range": f"{returns_series.index[0]} to {returns_series.index[-1]}"
            },
            tags=["ACF", "PACF", "ARIMA_parameter_calcualtion"]
        )

    def compute_acf_pacf(self) -> Dict[str, np.ndarray]:
        acf_values = acf(self.returns, nlags=self.max_lags, fft=True)
        pacf_values = pacf(self.returns, nlags=self.max_lags, method='yw')

        # Create a DataFrame for all computed values
        values_df = pd.DataFrame({
            'Lag': range(len(acf_values)),
            'ACF': acf_values,
            'PACF': pacf_values
        })

        # Log as table instead of individual values
        wandb.log({
            'ACF_PACF_Values': wandb.Table(dataframe=values_df.round(4))
        })

        return {
            self.ACF_KEY: acf_values,
            self.PACF_KEY: pacf_values
        }

    def plot_acf_pacf(self, acf_values: np.ndarray, pacf_values: np.ndarray, max_display_lags: int = 50) -> None:
        try:
            plt.figure(figsize=(15, 6))
            confidence_interval = 1.96 / np.sqrt(len(self.returns))

            # ACF Plot
            plt.subplot(1, 2, 1)
            plt.stem(range(min(len(acf_values), max_display_lags)),
                    acf_values[:max_display_lags])
            plt.axhline(y=confidence_interval, color='r', linestyle='--', alpha=0.5)
            plt.axhline(y=-confidence_interval, color='r', linestyle='--', alpha=0.5)
            plt.fill_between(range(max_display_lags),
                            confidence_interval,
                            -confidence_interval,
                            color='gray',
                            alpha=0.2)
            plt.title('Autocorrelation Function (ACF)')
            plt.xlabel('Lag')
            plt.ylabel('ACF')
            plt.grid(True, alpha=0.3)

            # PACF Plot
            plt.subplot(1, 2, 2)
            plt.stem(range(min(len(pacf_values), max_display_lags)),
                    pacf_values[:max_display_lags])
            plt.axhline(y=confidence_interval, color='r', linestyle='--', alpha=0.5)
            plt.axhline(y=-confidence_interval, color='r', linestyle='--', alpha=0.5)
            plt.fill_between(range(max_display_lags),
                            confidence_interval,
                            -confidence_interval,
                            color='gray',
                            alpha=0.2)
            plt.title('Partial Autocorrelation Function (PACF)')
            plt.xlabel('Lag')
            plt.ylabel('PACF')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            wandb.log({'ACF_PACF_Plots': wandb.Image(plt)})
            plt.close()
        finally:
            plt.close('all')


    def find_significant_lags(self, acf_values, pacf_values, significance_level=0.05):
      confidence_interval = 1.96 / np.sqrt(len(self.returns))
      significant_acf_lags = [i for i, val in enumerate(acf_values) if abs(val) > confidence_interval]
      significant_pacf_lags = [i for i, val in enumerate(pacf_values) if abs(val) > confidence_interval]

      # Create detailed DataFrame for ACF significant lags
      acf_sig_df = pd.DataFrame({
          'Lag': significant_acf_lags,
          'ACF_Value': [acf_values[i] for i in significant_acf_lags],
          'Normalized_ACF': [abs(acf_values[i]) / confidence_interval for i in significant_acf_lags],
          'Is_Highly_Significant': [abs(acf_values[i]) / confidence_interval > 2 for i in significant_acf_lags]
      }).sort_values('Normalized_ACF', ascending=False)

      # Create detailed DataFrame for PACF significant lags
      pacf_sig_df = pd.DataFrame({
          'Lag': significant_pacf_lags,
          'PACF_Value': [pacf_values[i] for i in significant_pacf_lags],
          'Normalized_PACF': [abs(pacf_values[i]) / confidence_interval for i in significant_pacf_lags],
          'Is_Highly_Significant': [abs(pacf_values[i]) / confidence_interval > 2 for i in significant_pacf_lags]
      }).sort_values('Normalized_PACF', ascending=False)

      # Log detailed tables to wandb
      wandb.log({
          'ACF_Significant_Lags_Detail': wandb.Table(dataframe=acf_sig_df.round(4)),
          'PACF_Significant_Lags_Detail': wandb.Table(dataframe=pacf_sig_df.round(4))
      })

      # Create and log summary table
      summary_df = pd.DataFrame({
          'Type': ['ACF', 'PACF'],
          'Count': [len(significant_acf_lags), len(significant_pacf_lags)],
          'Highly_Significant_Count': [sum(acf_sig_df['Is_Highly_Significant']),
                                    sum(pacf_sig_df['Is_Highly_Significant'])],
          'Max_Normalized_Value': [acf_sig_df['Normalized_ACF'].max() if not acf_sig_df.empty else 0,
                                pacf_sig_df['Normalized_PACF'].max() if not pacf_sig_df.empty else 0],
          'Significant_Lags_List': [str(significant_acf_lags), str(significant_pacf_lags)]
      })

      wandb.log({
          'Significant_Lags_Summary': wandb.Table(dataframe=summary_df)
      })

      return {
          self.ACF_LAGS_KEY: significant_acf_lags,
          self.PACF_LAGS_KEY: significant_pacf_lags
      }



    def suggest_arima_orders(self, acf_values: np.ndarray, pacf_values: np.ndarray,
                        significant_lags: Dict[str, List[int]], d: int = 0) -> Dict[str, Tuple[int, int, int]]:
      try:
          suggestions = {}

          # [Previous validation code remains the same]
          if not isinstance(significant_lags, dict) or self.ACF_LAGS_KEY not in significant_lags or self.PACF_LAGS_KEY not in significant_lags:
              raise ValueError(f"significant_lags must be a dictionary with '{self.ACF_LAGS_KEY}' and '{self.PACF_LAGS_KEY}' keys")

          acf_sig_lags = sorted(significant_lags.get(self.ACF_LAGS_KEY, []))
          pacf_sig_lags = sorted(significant_lags.get(self.PACF_LAGS_KEY, []))

          # Calculate all suggestions as before
          p_conservative = next((lag for lag in pacf_sig_lags if lag > 0), 0)
          q_conservative = next((lag for lag in acf_sig_lags if lag > 0), 0)
          suggestions['conservative'] = (p_conservative, d, q_conservative)

          p_moderate = min(len([lag for lag in pacf_sig_lags if 0 < lag <= 10]), 3)
          q_moderate = min(len([lag for lag in acf_sig_lags if 0 < lag <= 10]), 3)
          suggestions['moderate'] = (p_moderate, d, q_moderate)

          seasonal_p = self._find_seasonal_pattern(pacf_values)
          seasonal_q = self._find_seasonal_pattern(acf_values)

          p_aggressive = min(max(p_moderate + 1, seasonal_p), 5)
          q_aggressive = min(max(q_moderate + 1, seasonal_q), 5)
          suggestions['aggressive'] = (p_aggressive, d, q_aggressive)

          if seasonal_p > 0 or seasonal_q > 0:
              suggestions['seasonal'] = (
                  seasonal_p if seasonal_p > 0 else 1,
                  d,
                  seasonal_q if seasonal_q > 0 else 1
              )

          # Create a single DataFrame for all ARIMA suggestions
          suggestions_df = pd.DataFrame([
            {
                'Approach': k.capitalize(),
                'p': v[0],
                'd': v[1],
                'q': v[2],
                'ARIMA_Order': f'ARIMA({v[0]},{v[1]},{v[2]})',
                'Description': self._get_approach_description(k, v[0], v[1], v[2]),
                'Complexity': self._get_complexity_score(v[0], v[1], v[2])
            }
            for k, v in suggestions.items()
          ])

          # Sort by complexity
          suggestions_df = suggestions_df.sort_values('Complexity')

          # Log enhanced table to wandb
          wandb.log({
              'ARIMA_Suggestions_Detail': wandb.Table(dataframe=suggestions_df)
          })

          return suggestions

      except Exception as e:
          print(f"Error in suggest_arima_orders: {e}")
          return {'conservative': (1, d, 1)}

    def _get_approach_description(self, approach, p, d, q):
      descriptions = {
          'conservative': 'Minimal model using first significant lags',
          'moderate': 'Balanced model using first 10 lags',
          'aggressive': 'Complex model considering more lags',
          'seasonal': 'Model accounting for seasonal patterns'
      }
      return descriptions.get(approach.lower(), 'Custom approach')

    def _get_complexity_score(self, p, d, q):
      return p + d + q  # Simple complexity score


    def _find_pattern_cutoff(self, values: np.ndarray, threshold: float = 0.1) -> int:
        """Find where the ACF/PACF pattern cuts off"""
        for i in range(1, len(values)-1):
            if abs(values[i]) < threshold and abs(values[i+1]) < threshold:
                return i
        return min(5, len(values)-1)  # Default to 5 if no clear cutoff



    def _find_seasonal_pattern(self, values: np.ndarray) -> int:
        """Identify potential seasonal patterns in ACF/PACF"""
        peaks = []
        for i in range(2, len(values)-1):
            if (values[i] > values[i-1] and values[i] > values[i+1] and
                abs(values[i]) > 0.1):
                peaks.append(i)

        if len(peaks) >= 2:
            # Look for regular spacing between peaks
            differences = np.diff(peaks)
            if len(set(differences)) == 1:
                return differences[0]
        return 0



    def run_analysis(self, d=0):
      """
      Run the complete analysis workflow.
      """
      try:
          # Start timing
          start_time = datetime.now()

          # Get system info
          import psutil
          import platform
          import sys

          # Compute ACF and PACF
          acf_pacf_values = self.compute_acf_pacf()

          # Plot ACF and PACF
          self.plot_acf_pacf(acf_pacf_values[self.ACF_KEY], acf_pacf_values[self.PACF_KEY])

          # Find significant lags
          significant_lags = self.find_significant_lags(
              acf_pacf_values[self.ACF_KEY],
              acf_pacf_values[self.PACF_KEY]
          )

          # Get ARIMA suggestions
          arima_suggestions = self.suggest_arima_orders(
              acf_pacf_values[self.ACF_KEY],
              acf_pacf_values[self.PACF_KEY],
              significant_lags,
              d=d
          )

          # Calculate execution time
          execution_time = (datetime.now() - start_time).total_seconds()

          # Get system metrics
          system_metrics = pd.DataFrame([{
              'CPU_Usage_Percent': psutil.cpu_percent(),
              'Memory_Usage_Percent': psutil.virtual_memory().percent,
              'Available_Memory_GB': round(psutil.virtual_memory().available / (1024**3), 2),
              'Total_Memory_GB': round(psutil.virtual_memory().total / (1024**3), 2),
              'Python_Version': platform.python_version(),
              'OS_Platform': platform.platform(),
              'CPU_Cores': psutil.cpu_count(),
              'Execution_Time_Seconds': execution_time
          }])

          # Create comprehensive final summary
          all_stats_summary = pd.DataFrame([{
              'Total_ACF_Lags': len(significant_lags[self.ACF_LAGS_KEY]),
              'Total_PACF_Lags': len(significant_lags[self.PACF_LAGS_KEY]),
              'Number_of_ARIMA_Models': len(arima_suggestions),
              'Input_Series_Length': len(self.returns),
              'Differencing_Order': d,
              'Max_Lag_Analyzed': self.max_lags,
              'Data_Range': f"{self.returns.index[0]} to {self.returns.index[-1]}",
              'Analysis_Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
              'Execution_Time_Seconds': execution_time
          }])

          # Log summaries to wandb
          wandb.log({
              'Analysis_Complete_Summary': wandb.Table(dataframe=all_stats_summary),
              'System_Performance_Metrics': wandb.Table(dataframe=system_metrics)
          })

          # Log system metrics directly for tracking
          wandb.log({
              'CPU_Usage': psutil.cpu_percent(),
              'Memory_Usage': psutil.virtual_memory().percent,
              'Execution_Time': execution_time
          })

          # Finish wandb run
          wandb.finish()

          return {
              'significant_lags': significant_lags,
              'acf_pacf_values': acf_pacf_values,
              'arima_suggestions': arima_suggestions
          }

      except Exception as e:
          print(f"Error in run_analysis: {e}")
          wandb.finish()
          raise e

In [ ]:
# @title
analysis = TimeSeriesAnalysis(
            returns_series=df0,
            max_lags=200
        )

        # Run the complete analysis
results = analysis.run_analysis()

In [ ]:



def bai_perron_test(data, max_breaks=5, min_size=0.1, early_stopping_patience=3):
    """
    Enhanced Bai-Perron test with comprehensive time tracking and GPU optimization
    """
    logger = setup_logger()

    # Initialize timing dictionary
    timings = {}
    total_start = time.time()

    with cp.cuda.Device(0):  # Explicitly select GPU device
        try:
            # Track overall process with main progress bar
            with tqdm(total=3, desc="Overall Progress", position=0, leave=True) as main_pbar:

                # Stage 1: Initialization and Data Transfer
                init_start = time.time()
                with tqdm(total=4, desc="GPU Initialization", position=1, leave=False) as init_pbar:
                    # Move data to GPU
                    data_transfer_start = time.time()
                    data_gpu = cp.asarray(data)
                    n = len(data_gpu)
                    min_size_obs = int(min_size * n)
                    print("####### min_size_obs = " + str(min_size_obs))
                    timings['data_transfer'] = time.time() - data_transfer_start
                    init_pbar.update(1)

                    # Create time trend on GPU
                    matrix_start = time.time()
                    time_trend = cp.arange(n)
                    X_gpu = cp.column_stack((cp.ones(n), time_trend))
                    timings['matrix_creation'] = time.time() - matrix_start
                    init_pbar.update(1)

                    # Initialize arrays
                    array_start = time.time()
                    rss_array = cp.zeros((n, max_breaks+1))
                    break_points = cp.zeros((n, max_breaks+1), dtype=int)
                    aic_scores = cp.zeros(max_breaks+1)
                    timings['array_initialization'] = time.time() - array_start
                    init_pbar.update(1)

                    # Base case calculations
                    base_case_start = time.time()
                    beta_base = cp.linalg.pinv(X_gpu) @ data_gpu
                    residuals = data_gpu - X_gpu @ beta_base
                    base_rss = float(cp.sum(residuals**2))
                    r2 = 1 - base_rss / cp.sum((data_gpu - cp.mean(data_gpu))**2)

                    rss_array[0, 0] = base_rss
                    aic_scores[0] = calculate_aic(base_rss, 2, n)

                    wandb.log({
                        'base_case_rss': float(base_rss),
                        'base_case_r2': float(r2),
                        'base_case_aic': float(aic_scores[0])
                    })
                    timings['base_case'] = time.time() - base_case_start
                    init_pbar.update(1)

                timings['initialization'] = time.time() - init_start
                main_pbar.update(1)

                # Stage 2: Break Detection
                break_detection_start = time.time()
                best_aic = float('inf')
                no_improve = 0

                # Find breaks using AIC with nested progress bars
                with tqdm(range(1, max_breaks+1),
                         desc="Break Detection",
                         position=1,
                         leave=False) as break_pbar:

                    for m in break_pbar:
                        segment_start = time.time()
                        segment_metrics = []
                        min_aic = float('inf')
                        best_break = None

                        # Inner loop for segment search
                        print("####### starting inner loop for segment search, min_size_obs = " + str(min_size_obs))
                        with tqdm(range(min_size_obs, n-min_size_obs+1),
                                desc=f"Break {m}/{max_breaks}",
                                position=2,
                                leave=False) as segment_pbar:

                            print("####### segment_pbar = " + str(segment_pbar))
                            for i in segment_pbar:
                                # Split data and fit models
                                y1, y2 = data_gpu[:i], data_gpu[i:]
                                X1, X2 = X_gpu[:i], X_gpu[i:]

                                beta1 = cp.linalg.pinv(X1) @ y1
                                beta2 = cp.linalg.pinv(X2) @ y2

                                residuals1 = y1 - X1 @ beta1
                                residuals2 = y2 - X2 @ beta2

                                rss = float(cp.sum(residuals1**2) + cp.sum(residuals2**2))
                                rss_array[i, m] = rss
                                current_aic = calculate_aic(rss, 2*(m+1), n)

                                if current_aic < min_aic:
                                    min_aic = current_aic
                                    best_break = i
                                    r2_1 = 1 - cp.sum(residuals1**2) / cp.sum((y1 - cp.mean(y1))**2)
                                    r2_2 = 1 - cp.sum(residuals2**2) / cp.sum((y2 - cp.mean(y2))**2)

                                    segment_metrics = [{
                                        'segment': m,
                                        'break_point': i,
                                        'rss': float(rss),
                                        'r2_before': float(r2_1),
                                        'r2_after': float(r2_2),
                                        'aic': float(current_aic)
                                    }]

                                # Update progress metrics
                                elapsed = time.time() - segment_start
                                segment_pbar.set_postfix({
                                    'Current AIC': f"{current_aic:.2f}",
                                    'Best AIC': f"{min_aic:.2f}",
                                    'Time': f"{elapsed:.1f}s"
                                })

                        # Early stopping check
                        if min_aic > best_aic:
                            no_improve += 1
                        else:
                            best_aic = min_aic
                            no_improve = 0

                        if no_improve >= early_stopping_patience:
                            print(f"\nEarly stopping at {m} breaks")
                            break

                        if best_break is not None:
                            break_points[:, m] = best_break
                            aic_scores[m] = min_aic

                            wandb.log({
                                f'break_{m}_metrics': segment_metrics,
                                f'aic_score_{m}': float(min_aic),
                                f'break_{m}_time': time.time() - segment_start
                            })

                        break_pbar.set_postfix({
                            'Best AIC': f"{min_aic:.2f}",
                            'Time': f"{time.time() - break_detection_start:.1f}s"
                        })

                timings['break_detection'] = time.time() - break_detection_start
                main_pbar.update(1)

                # Stage 3: Results Processing
                results_start = time.time()

                # Find optimal breaks
                optimal_breaks = int(cp.argmin(aic_scores))
                break_dates = sorted([
                    int(break_points[i, i+1])
                    for i in range(optimal_breaks)
                ])

                # Move arrays back to CPU for plotting
                aic_scores_cpu = cp.asnumpy(aic_scores)

                # Plot AIC scores
                plt.figure(figsize=(10, 6))
                plt.plot(range(max_breaks+1), aic_scores_cpu, 'o-')
                plt.xlabel('Number of Breaks')
                plt.ylabel('AIC Score')
                plt.title('AIC Scores vs Number of Breaks')

                # Log final results
                timings['results_processing'] = time.time() - results_start
                timings['total_time'] = time.time() - total_start

                wandb.log({
                    "aic_plot": wandb.Image(plt),
                    "optimal_breaks": optimal_breaks,
                    "aic_scores": aic_scores_cpu.tolist(),
                    "break_dates": break_dates,
                    "execution_times": timings,
                    "memory_usage": cp.get_default_memory_pool().used_bytes()
                })

                plt.close()
                main_pbar.update(1)

                return break_dates, aic_scores_cpu

        finally:
            # Clean up GPU memory
            mempool = cp.get_default_memory_pool()
            pinned_mempool = cp.get_default_pinned_memory_pool()
            mempool.free_all_blocks()
            pinned_mempool.free_all_blocks()



def analyze_regime_characteristics(data, break_dates):
    print("break_dates as argument at line 287: " + break_dates)
    """
    Analyze characteristics of each regime and store in DataFrames
    """
    start_time = time.time()

    # Initialize lists to store data for DataFrames
    regime_data = []
    regime_returns_data = []

    with tqdm(total=len(break_dates) + 1, desc="Analyzing Regimes") as pbar:
        start_idx = 0

        for regime_num, break_idx in enumerate(break_dates + [len(data)], 1):
            print("INSIDE loop at line 300")
            # Extract regime segment
            segment = data[start_idx:break_idx]
            returns = pd.Series(segment).pct_change().dropna()

            # Basic regime statistics
            regime_stats = {
                'regime_number': regime_num,
                'start_idx': start_idx,
                'end_idx': break_idx,
                'length': len(segment),
                'mean': np.mean(segment),
                'std': np.std(segment),
                'skew': pd.Series(segment).skew(),
                'kurtosis': pd.Series(segment).kurtosis(),
                'min': np.min(segment),
                'max': np.max(segment),
                'median': np.median(segment),
                'volatility_5d': pd.Series(segment).rolling(5).std().mean(),
                'volatility_20d': pd.Series(segment).rolling(20).std().mean()
            }

            # Return statistics
            return_stats = {
                'regime_number': regime_num,
                'mean_return': returns.mean(),
                'std_return': returns.std(),
                'positive_returns_pct': (returns > 0).mean() * 100,
                'negative_returns_pct': (returns < 0).mean() * 100,
                'max_return': returns.max(),
                'min_return': returns.min()
            }

            regime_data.append(regime_stats)
            regime_returns_data.append(return_stats)

            start_idx = break_idx
            pbar.update(1)

    # Create DataFrames
    regimes_df = pd.DataFrame(regime_data)
    returns_df = pd.DataFrame(regime_returns_data)

    # Log to wandb as tables
    wandb.log({
        "regime_statistics": wandb.Table(dataframe=regimes_df),
        "regime_returns": wandb.Table(dataframe=returns_df),
        "analysis_time": time.time() - start_time,
        "total_regimes": len(regimes_df)
    })
    return regimes_df, returns_df



def plot_regimes(data, break_dates, regime_stats):
    """
    Plot time series regimes with statistical annotations and volatility.

    Parameters:
    -----------
    data : array-like
        Time series data
    break_dates : list
        Indices of structural break points
    regime_stats : tuple or list
        Regime statistical information

    Returns:
    --------
    fig : matplotlib.figure.Figure
        Generated visualization
    """
    logger = setup_logger()

    try:
        # Validate inputs
        if not isinstance(data, (np.ndarray, pd.Series, list)):
            raise ValueError("Data must be a numpy array, pandas Series, or list")

        # Replace logger warnings with print statements or wandb logging
        if not break_dates:
            print("Warning: No break dates provided. Plotting entire series as single regime.")
            break_dates = []

        if not regime_stats:
            print("Warning: No regime statistics provided. Creating default statistics.")
            regime_stats = [{'mean': np.mean(data), 'std': np.std(data)} for _ in range(len(break_dates) + 1)]

        # Unpack regime statistics
        if isinstance(regime_stats, tuple):
            regimes_df, returns_df = regime_stats
            regime_stats = regimes_df.to_dict('records')


        # Create figure
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), height_ratios=[3, 1])

        # Generate colors for regimes
        colors = plt.cm.rainbow(np.linspace(0, 1, len(regime_stats)))

        # Plot each regime with different color
        start_idx = 0
        for i, (end_idx, color) in enumerate(zip(break_dates + [len(data)], colors)):
            try:
                regime_data = data[start_idx:end_idx]
                regime_index = range(start_idx, end_idx)

                # Main price plot with regime coloring
                ax1.plot(regime_index, regime_data, color=color, label=f'Regime {i+1}')

                # Add regime statistics as text
                stats = regime_stats[i]
                mid_point = start_idx + len(regime_data)//2

                # Safely access mean and std
                mean = stats.get('mean', stats.get('Mean', 0))
                std = stats.get('std', stats.get('Std', 0))

                ax1.text(mid_point, ax1.get_ylim()[1],
                         f'R{i+1}\nμ={mean:.2e}\nσ={std:.2e}',
                         horizontalalignment='center', alpha=0.7)

                # Volatility subplot with safety check
                rolling_window = min(75, max(3, len(regime_data) // 2))
                volatility = (pd.Series(regime_data).rolling(rolling_window, min_periods=1).std()
                              if len(regime_data) > rolling_window
                              else pd.Series(np.std(regime_data) * np.ones(len(regime_data))))

                ax2.plot(regime_index, volatility, color=color, alpha=0.7)

                start_idx = end_idx

            except Exception as regime_error:
                logger.error(f"Error processing regime {i+1}: {regime_error}")
                continue

        # Add vertical lines at break points
        for date in break_dates:
            ax1.axvline(x=date, color='black', linestyle='--', alpha=0.3)
            ax2.axvline(x=date, color='black', linestyle='--', alpha=0.3)

        # Styling and labels
        ax1.set_title('Time Series with Regime Breaks and Statistics')
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax1.set_ylabel('Price')

        ax2.set_title('Rolling Volatility by Regime')
        ax2.set_xlabel('Time Index')
        ax2.set_ylabel('Volatility')

        plt.tight_layout()

        # Log plot to wandb
        try:
            wandb.log({"detailed_regime_plot": wandb.Image(fig)})
        except Exception as wandb_error:
            logger.warning(f"Could not log plot to wandb: {wandb_error}")

        plt.close()

        return fig

    except Exception as main_error:
        logger.error(f"Critical error in plot_regimes: {main_error}")
        return None


def process_single_combination(args):
    print("Inside process_single_combination")
    """GPU-optimized helper function for parallel processing"""
    data, max_breaks, days = args
    # Log processing configuration to wandb
    wandb.log({
        "processing_config": {
            "max_breaks": max_breaks,
            "min_size_days": days
        }
    })

    try:
        # Validate inputs
        if not isinstance(data, (np.ndarray, pd.Series, list)):
            raise ValueError("Input data must be a numpy array, pandas Series, or list")

        # Progress tracking
        with tqdm(total=5, desc=f"Processing (max_breaks={max_breaks}, days={days})", position=0, leave=True) as main_pbar:
            main_pbar.set_description(f"Initializing GPU Processing")

            with cp.cuda.Device(0):  # Ensure GPU context
                try:
                    cp.cuda.Stream.null.synchronize()  # Synchronize GPU operations
                    main_pbar.update(1)

                    # Move data to GPU
                    main_pbar.set_description("Transferring data to GPU")
                    data_gpu = cp.asarray(data)
                    min_size = convert_days_to_periods(days) / len(data)
                    main_pbar.update(1)

                    # Run Bai-Perron test
                    main_pbar.set_description("Running Structural Break Analysis")
                    break_dates, aic_scores = bai_perron_test(data_gpu, max_breaks, min_size)
                    main_pbar.update(1)

                    # Move data back to CPU
                    main_pbar.set_description("Transferring results to CPU")
                    data_cpu = cp.asnumpy(data_gpu)
                    main_pbar.update(1)

                    # Clean GPU memory
                    main_pbar.set_description("Cleaning GPU Memory")
                    mempool = cp.get_default_memory_pool()
                    pinned_mempool = cp.get_default_pinned_memory_pool()
                    mempool.free_all_blocks()
                    pinned_mempool.free_all_blocks()
                    main_pbar.update(1)

                except cp.cuda.runtime.CUDARuntimeError as e:
                    wandb.log({
                        "gpu_error": {
                            "message": str(e),
                            "fallback": "CPU processing"
                        }
                    })
                    print(f"GPU Error: {e}")
                    print("Falling back to CPU processing...")

                    # CPU Fallback
                    min_size = convert_days_to_periods(days) / len(data)
                    break_dates, aic_scores = bai_perron_test(data, max_breaks, min_size)
                    data_cpu = data

            # Regime analysis and visualization
            regime_stats = analyze_regime_characteristics(data_cpu, break_dates)
            plot_regimes(data_cpu, break_dates, regime_stats)

            # Log results to wandb
            wandb.log({
                "processing_results": {
                    "max_breaks": max_breaks,
                    "min_size_days": days,
                    "n_breaks": len(break_dates),
                    "min_aic": float(np.min(aic_scores)) if len(aic_scores) > 0 else None
                }
            })
            print("returning from process_single_combination")

            return break_dates, aic_scores, regime_stats

    except Exception as e:
        # Comprehensive error logging
        wandb.log({
            "critical_error": {
                "error_message": str(e),
                "error_type": type(e).__name__,
                "max_breaks": max_breaks,
                "min_size_days": days
            }
        })
        raise



def main(df0):
    """Main execution with optimizations"""
    param_grid = {
        'max_breaks': [20],  ## 40,60
        'min_size_days': [5]    ## 3,5,7
    }

    data = df0.copy()

    # Initialize wandb
    wandb.init(
        project="Time Series Analysis of Nifty50 5 min ohlc",
        name=f"Test---regime_detection_grid_search_{datetime.now().strftime('%Y%m%d_%H%M')}",
        group="structural_breaks_analysis",
        config={
            "data_points": len(data),
            "analysis_type": "Bai_Perron_Test",
            "max_breaks_tested": param_grid['max_breaks'],
            "min_size_days_tested": param_grid['min_size_days'],
            "data_range": f"from index {0} to {len(data)-1}"
        },
        tags=["structural_breaks", "regime_detection", "AIC_optimization"]
    )

    # Create DataFrames for storing results
    all_results = []
    all_regime_stats = []
    all_break_points = []

    # Prepare parameter combinations for parallel processing
    combinations = [
        (data, max_breaks, days)
        for max_breaks in param_grid['max_breaks']
        for days in param_grid['min_size_days']
    ]

    # Enhanced progress tracking
    total_combinations = len(combinations)
    print("at line 603")
    with tqdm(total=total_combinations,
              desc="Overall Progress",
              position=0,
              bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as main_progress:

        # Parallel processing with enhanced tracking
        print("at line 609")
        n_cores = max(1, multiprocessing.cpu_count() - 1)
        with ProcessPoolExecutor(max_workers=n_cores) as executor:
            # Use as_completed for better progress tracking
            future_to_params = {
                executor.submit(process_single_combination, combo): combo
                for combo in combinations
            }
            print("at line 616")
            results = []
            for future in as_completed(future_to_params):
                print("at line 622")
                combo = future_to_params[future]
                try:
                    print("at line 626")
                    result = future.result()
                    print("result at line 622 = " + result)
                    results.append(result)

                    # Update progress bar
                    main_progress.update(1)
                    main_progress.set_postfix({
                        'max_breaks': combo[1],
                        'min_size_days': combo[2]
                    })

                    # Log intermediate result
                    wandb.log({
                        "completed_config": {
                            "max_breaks": combo[1],
                            "min_size_days": combo[2],
                            "progress": len(results) / total_combinations
                        }
                    })

                except Exception as e:
                    traceback.print_exc()
                    wandb.log({
                        "processing_error": {
                            "combo": str(combo),
                            "error": str(e)
                        }
                    })
    print("##################")
    print("regime_stats = " + regime_stats)
    print("break_dates = " + break_dates)
    print("aic_scores = " + aic_scores)
    print("max_breaks = " + max_breaks)
    for (max_breaks, days), (break_dates, aic_scores, regime_stats) in zip(
        [(mb, d) for mb in param_grid['max_breaks'] for d in param_grid['min_size_days']],
        results
    ):
      # Store main results
        result_entry = {
            'max_breaks': max_breaks,
            'min_size_days': days,
            'n_breaks_found': len(break_dates),
            'min_aic': np.min(aic_scores),
            'mean_aic': np.mean(aic_scores),
            'std_aic': np.std(aic_scores),
            'n_regimes': len(regime_stats),
            'timestamp': datetime.now().strftime('%Y%m%d_%H%M')
        }
        all_results.append(result_entry)

        # Store regime statistics
        for i, stats in enumerate(regime_stats):
            print("INSIDE Store regime statistics loop")
            print("regime_stats = " + regime_stats)
            regime_entry = {
                'max_breaks': max_breaks,
                'min_size_days': days,
                'regime_number': i+1,
                'start_idx': stats['start_idx'],
                'end_idx': stats['end_idx'],
                'length': stats['length'],
                'mean_return': stats['mean'],
                'volatility': stats['std'],
                'skewness': stats['skew'],
                'kurtosis': stats['kurtosis']
            }
            all_regime_stats.append(regime_entry)

        # Store break points
        for i, break_point in enumerate(break_dates):
            break_entry = {
                'max_breaks': max_breaks,
                'min_size_days': days,
                'break_number': i+1,
                'break_point_index': break_point,
                'aic_score': aic_scores[i] if i < len(aic_scores) else None
            }
            all_break_points.append(break_entry)

        # Log current iteration results
        wandb.log({
            "grid_step": len(all_results),
            **result_entry
        })

    # Create final DataFrames
    results_df = pd.DataFrame(all_results)
    print("all_regime_stats")
    print(all_regime_stats)
    regimes_df = pd.DataFrame(all_regime_stats)
    breaks_df = pd.DataFrame(all_break_points)

    # Log final results
    wandb.log({
        "final_summary_table": wandb.Table(dataframe=results_df),
        "final_regime_statistics": wandb.Table(dataframe=regimes_df),
        "final_break_points": wandb.Table(dataframe=breaks_df),
        "completed_configurations": len(results_df)
    })

    # Save results locally
    results_df.to_csv('structural_breaks_summary.csv', index=False)
    regimes_df.to_csv('regime_statistics.csv', index=False)
    breaks_df.to_csv('break_points.csv', index=False)

    # Print summaries
    print("\nAnalysis Summary:")
    print(results_df)
    print("\nRegime Statistics:")
    print(regimes_df)
    print(regimes_df.groupby(['max_breaks', 'min_size_days'])['length'].describe())
    print("\nBreak Points Distribution:")
    print(breaks_df.groupby(['max_breaks', 'min_size_days']).size())

    wandb.finish()
    return results_df, regimes_df, breaks_df

def load_data_efficiently(df0):
    # Convert to float32 instead of float64
    data = df0.astype(np.float32)
    # Free original dataframe memory
    del df0
    gc.collect()
    return data


if __name__ == "__main__":
    df0 = load_data_efficiently(df0)
    results_df, regimes_df, breaks_df = main(df0)